# Import Files

In [2]:
from pathlib import Path

BASE_DIR = Path(r"D:\Kuliah\Semester 7\AISO\TUBES AISO\New")

DATA_YAML = BASE_DIR / "data.yaml"  
print("Base dir :", BASE_DIR)
print("data.yaml:", DATA_YAML.exists())

Base dir : D:\Kuliah\Semester 7\AISO\TUBES AISO\New
data.yaml: True


In [3]:
from ultralytics import YOLO
import os, sys, json
from pathlib import Path
print("Ultralytics version OK.")


Ultralytics version OK.


In [4]:
for sub in ["train/images", "valid/images", "test/images"]:
    p = BASE_DIR / sub
    if p.exists():
        sampel = sorted([f for f in p.glob("*") if f.suffix.lower() in {".jpg",".jpeg",".png",".bmp",".webp"}])[:5]
        print(f"{sub} -> {len(list(p.glob('*')))} files; contoh:", [s.name for s in sampel])
    else:
        print(f"Folder tidak ditemukan: {p}")

train/images -> 4370 files; contoh: ['0_jpg.rf.b9489066dff83207fd5360f7ca7d6950.jpg', '0_jpg.rf.c6fbe133d06e96bd68d2dea58cf2523d.jpg', '1000_F_110700166_utOk6hFlRPRAmIStyojORzbmTlfZOsh8_jpg.rf.ac9778142e0c97dc20d02ca376319562.jpg', '1000_F_3252303_fxYr0WTmv56GZMAbG4E1XzeJ6G4iWp_jpg.rf.266e7a3b55049220b3a55315b37d40c5.jpg', '1000_F_3252303_fxYr0WTmv56GZMAbG4E1XzeJ6G4iWp_jpg.rf.3da88c08f1709fe973a58f2f52c4e68f.jpg']
valid/images -> 1233 files; contoh: ['100car_jpg.rf.5f0e5a412e76945aaf744840da4f9604.jpg', '101Ambulance_jpg.rf.f97b5ceed62bb2637e2591f420871cda.jpg', '101car_jpg.rf.b98471406bb48d33367b24addda19f84.jpg', '102car_jpg.rf.ebd8e894ae46af2447b26401c473358e.jpg', '103_jpg.rf.34efd4fd23c28338f024d6247430c2c4.jpg']
test/images -> 587 files; contoh: ['100Truck_jpg.rf.6d1fb54756da5451afc5d936cce31de8.jpg', '103car_jpg.rf.a7d51ea114c795df35e5cab28e56c85f.jpg', '106Ambulance_jpg.rf.a7c62a2f4d83cfd74c1d7abd4f76b8b3.jpg', '107_jpg.rf.4d3fee9c1a63b6f71882bedb0e05dbc3.jpg', '107_jpg.rf.6d37

In [5]:
import yaml
with open(DATA_YAML, "r", encoding="utf-8") as f:
    y = yaml.safe_load(f)

print("Isi data.yaml:")
print(y)

Isi data.yaml:
{'train': '../train/images', 'val': '../valid/images', 'test': '../test/images', 'nc': 15, 'names': ['Ambulance', 'Auto', 'Bicycle', 'Bike', 'Bus', 'Car', 'Motorcycle', 'Truck', 'auto-rickshaw', 'bus', 'car', 'lorry', 'mini truck', 'motorcycle', 'truck'], 'roboflow': {'workspace': 'yolo-trials', 'project': 'vehicle-detection-eckrb', 'version': 4, 'license': 'CC BY 4.0', 'url': 'https://universe.roboflow.com/yolo-trials/vehicle-detection-eckrb/dataset/4'}}


In [6]:
import random, shutil

some_img = next((BASE_DIR/"valid/images").glob("*.*"))
print("Contoh gambar:", some_img)

Contoh gambar: D:\Kuliah\Semester 7\AISO\TUBES AISO\New\valid\images\100car_jpg.rf.5f0e5a412e76945aaf744840da4f9604.jpg


# Model Training

In [7]:
model = YOLO("yolov8n.pt")

results = model.train(
    data=str(DATA_YAML.as_posix()), 
    epochs=50,
    imgsz=640,
    batch=16,
    device=0 if YOLO().overrides.get('device', 'cpu') != 'cpu' else 'cpu' 
)

Ultralytics 8.3.223  Python-3.11.9 torch-2.9.0+cpu CPU (Intel Core Ultra 7 155H)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:/Kuliah/Semester 7/AISO/TUBES AISO/New/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0,

# Model Evaluation

In [ ]:
metrics = model.val(data=str(DATA_YAML.as_posix()), imgsz=640)
print("ringkas:", metrics.results_dict) 

# Model Testing

In [ ]:
from ultralytics import YOLO
from pathlib import Path

MODEL_PATH = Path(r"D:\Kuliah\Semester 7\AISO\TUBES AISO\runs\detect\train\weights\best.pt")
IMG_PATH = Path(r"D:\Kuliah\Semester 7\AISO\TUBES AISO\test_img3.jpg")

OUTPUT_DIR = r"D:\Kuliah\Semester 7\AISO\TUBES AISO\runs\detect\train\runs\detect\test_itb"

print("Model exists:", MODEL_PATH.exists())
print("Image exists:", IMG_PATH.exists())

model = YOLO(str(MODEL_PATH))

results = model.predict(
    source=str(IMG_PATH),
    imgsz=640,         
    conf=0.25,         
    iou=0.45,           
    save=True,          
    project=OUTPUT_DIR, 
    name="",            
    exist_ok=True,      
    device="cpu"       
)

print("✅ Hasil disimpan di:", OUTPUT_DIR)


Model exists: True
Image exists: True

image 1/1 D:\Kuliah\Semester 7\AISO\TUBES AISO\test_img3.jpg: 384x640 1 motorcycle, 63.0ms
Speed: 1.9ms preprocess, 63.0ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to D:\Kuliah\Semester 7\AISO\TUBES AISO\runs\detect\train\runs\detect\test_itb\predict
✅ Hasil disimpan di: D:\Kuliah\Semester 7\AISO\TUBES AISO\runs\detect\train\runs\detect\test_itb


In [46]:
from ultralytics import YOLO
from pathlib import Path

# === PATH SETUP ===
MODEL_PATH = Path(r"D:\Kuliah\Semester 7\AISO\TUBES AISO\runs\detect\train\weights\best.pt")
VIDEO_PATH = Path(r"D:\Kuliah\Semester 7\AISO\TUBES AISO\test_vid3.mp4")
OUTPUT_DIR = r"D:\Kuliah\Semester 7\AISO\TUBES AISO\runs\detect\train\runs\detect\test_itb"

print("Model exists:", MODEL_PATH.exists())
print("Video exists:", VIDEO_PATH.exists())

model = YOLO(str(MODEL_PATH))

results = model.predict(
    source=str(VIDEO_PATH),   
    imgsz=640,             
    conf=0.25,           
    iou=0.45,                
    save=True,              
    save_txt=False,           
    project=OUTPUT_DIR,       
    name="",                 
    exist_ok=True,           
    device="cpu"            
)

print("✅ Hasil video tersimpan di:", OUTPUT_DIR)

from pathlib import Path

out_dir = Path(OUTPUT_DIR)
out_videos = list(out_dir.glob("*.mp4")) + list(out_dir.glob("*.avi")) + list(out_dir.glob("*.mkv"))
print("File video output:", [v.name for v in out_videos])


Model exists: True
Video exists: True

WARNING 
inference results will accumulate in RAM unless `stream=True` is passed, causing potential out-of-memory
errors for large sources or long-running streams and videos. See https://docs.ultralytics.com/modes/predict/ for help.

Example:
    results = model(source=..., stream=True)  # generator of Results objects
    for r in results:
        boxes = r.boxes  # Boxes object for bbox outputs
        masks = r.masks  # Masks object for segment masks outputs
        probs = r.probs  # Class probabilities for classification outputs

video 1/1 (frame 1/164) D:\Kuliah\Semester 7\AISO\TUBES AISO\test_vid3.mp4: 640x384 2 cars, 1 motorcycle, 50.8ms
video 1/1 (frame 2/164) D:\Kuliah\Semester 7\AISO\TUBES AISO\test_vid3.mp4: 640x384 2 cars, 1 motorcycle, 47.0ms
video 1/1 (frame 3/164) D:\Kuliah\Semester 7\AISO\TUBES AISO\test_vid3.mp4: 640x384 2 cars, 75.1ms
video 1/1 (frame 4/164) D:\Kuliah\Semester 7\AISO\TUBES AISO\test_vid3.mp4: 640x384 2 cars, 48.8

# Model Testing (Pre-trained)

In [52]:
from ultralytics import YOLO
from pathlib import Path


MODEL_PATH = "yolov8n.pt"   
SOURCE_PATH = Path(r"D:\Kuliah\Semester 7\AISO\TUBES AISO\test_img3.jpg") 
OUTPUT_DIR = r"D:\Kuliah\Semester 7\AISO\TUBES AISO\runs\detect\train\runs\detect\test_itb\predict\test_itb_pretrained"

print("Source exists:", SOURCE_PATH.exists())


model = YOLO(MODEL_PATH)  


results = model.predict(
    source=str(SOURCE_PATH),   
    imgsz=640,
    conf=0.25,                
    iou=0.45,
    save=True,
    project=OUTPUT_DIR,        
    name="",                  
    exist_ok=True,
    device="cpu"               
)

print("✅ Hasil tersimpan di:", OUTPUT_DIR)


Source exists: True

image 1/1 D:\Kuliah\Semester 7\AISO\TUBES AISO\test_img3.jpg: 384x640 1 person, 2 motorcycles, 61.4ms
Speed: 1.6ms preprocess, 61.4ms inference, 0.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to D:\Kuliah\Semester 7\AISO\TUBES AISO\runs\detect\train\runs\detect\test_itb\predict\test_itb_pretrained\predict
✅ Hasil tersimpan di: D:\Kuliah\Semester 7\AISO\TUBES AISO\runs\detect\train\runs\detect\test_itb\predict\test_itb_pretrained
